# EDA - Simplified Version

## Key Features:
1. `event_key` parsed directly from filename
2. Snow pixels are **included** in images, but tracked via `snow_frac`
3. Can filter by snow fraction threshold in analysis
4. Useful for winter storm events (e.g., 2021 Texas)

In [ ]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, box
from rasterio.mask import mask

# 0) Path Configuration

In [ ]:
BASE = Path("..")  # project/
DATA = BASE / "data"
RAW_IMGS = DATA / "raw" / "imgs"
PROCESSED = DATA / "processed"
RESULT = BASE / "result"
RESULT.mkdir(parents=True, exist_ok=True)

PLOT_DIR = RESULT / "plots"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

# New v2 folders
IMG_DIRS = [
    RAW_IMGS / "GEE_VNP46A2_outages_v2_Dallas",
    RAW_IMGS / "GEE_VNP46A2_outages_v2_Harris",
]

CLOUD_SUMMARY = RAW_IMGS / "GEE_VNP46A2_outages_tables" / "cloud_fraction_summary_v2.csv"

# Parameters
PRE_DAYS = 5
POST_DAYS = 5
CLOUD_THRESHOLD = 0.3

# NEW: Snow filtering threshold
# Set to None to include all images regardless of snow
# Set to a value (e.g., 0.5) to exclude images with snow_frac > threshold
SNOW_THRESHOLD = None  # None = keep all, 0.5 = exclude if >50% snow

# 1) Read cloud summary

In [ ]:
cloud = pd.read_csv(CLOUD_SUMMARY, parse_dates=["date"])
cloud["date"] = pd.to_datetime(cloud["date"]).dt.normalize()

# Calculate snow fraction
cloud["snow_frac"] = cloud["snow_px"] / cloud["base_valid_px"].clip(lower=1)

# Mark usable images (cloud filter)
cloud["usable"] = (
    (cloud["img_exists"] == True) &
    (cloud["cloud_frac"] >= 0) &
    (cloud["cloud_frac"] <= CLOUD_THRESHOLD) &
    (cloud["base_valid_px"] > 0)
)

# Optional: Apply snow filter
if SNOW_THRESHOLD is not None:
    cloud["usable"] = cloud["usable"] & (cloud["snow_frac"] <= SNOW_THRESHOLD)
    print(f"Snow threshold applied: <= {SNOW_THRESHOLD}")
else:
    print("Snow threshold: None (all snow levels included)")

print(f"\nCloud summary records: {len(cloud)}")
print(f"Usable records: {cloud['usable'].sum()}")

# Show snow statistics
print(f"\nSnow fraction statistics:")
print(cloud["snow_frac"].describe())

# 2) Parse outage dates from event_key

In [ ]:
def parse_event_key(event_key):
    """
    Parse event_key to extract county and outage dates
    Format: {county}_{startYYYYMMDDHHMM}_{endYYYYMMDDHHMM}
    """
    parts = event_key.split("_")
    county = parts[0]
    start_str = parts[1]
    end_str = parts[2]
    
    outage_start = pd.to_datetime(start_str, format="%Y%m%d%H%M").normalize()
    outage_end = pd.to_datetime(end_str, format="%Y%m%d%H%M").normalize()
    
    return county, outage_start, outage_end

# Apply parsing
parsed = cloud["event_key"].apply(parse_event_key)
cloud["county_parsed"] = parsed.apply(lambda x: x[0])
cloud["outage_start"] = parsed.apply(lambda x: x[1])
cloud["outage_end"] = parsed.apply(lambda x: x[2])

print("Parsed outage dates from event_key")
display(cloud[["event_key", "county", "outage_start", "outage_end"]].drop_duplicates().head())

# 3) Scan local images

In [ ]:
# Filename pattern: {event_key}_{YYYYMMDD}_VNP46A2_ntl.tif
pat = re.compile(
    r"^([A-Za-z]+_\d{12}_\d{12})_(\d{8})_VNP46A2_ntl\.tif$", 
    re.I
)

img_records = []
for d in IMG_DIRS:
    if not d.exists():
        print(f"Warning: {d} does not exist")
        continue
    for fp in d.glob("*.tif"):
        m = pat.match(fp.name)
        if not m:
            continue
        event_key = m.group(1)
        date = pd.to_datetime(m.group(2), format="%Y%m%d").normalize()
        img_records.append({
            "event_key": event_key,
            "date": date,
            "path": fp
        })

imgs = pd.DataFrame(img_records)
print(f"Found {len(imgs)} images")

if imgs.empty:
    raise FileNotFoundError("No matching GeoTIFFs found")

# 4) Merge with cloud summary

In [ ]:
cloud_usable = cloud.loc[
    cloud["usable"], 
    ["event_key", "date", "county", "event_id", "outage_start", "outage_end", 
     "snow_px", "base_valid_px", "snow_frac"]
].copy()

imgs = imgs.merge(cloud_usable, on=["event_key", "date"], how="inner")
print(f"Images after merge: {len(imgs)}")

# 5) Phase labeling

In [ ]:
def label_phase(row):
    pre_start = row["outage_start"] - pd.Timedelta(days=PRE_DAYS)
    pre_end = row["outage_start"] - pd.Timedelta(days=1)
    post_start = row["outage_end"] + pd.Timedelta(days=1)
    post_end = row["outage_end"] + pd.Timedelta(days=POST_DAYS)
    
    if pre_start <= row["date"] <= pre_end:
        return "pre"
    elif row["outage_start"] <= row["date"] <= row["outage_end"]:
        return "during"
    elif post_start <= row["date"] <= post_end:
        return "post"
    return None

imgs["phase"] = imgs.apply(label_phase, axis=1)
imgs = imgs[imgs["phase"].isin(["pre", "during", "post"])].copy()
print(f"Images after phase labeling: {len(imgs)}")

# 6) Calculate mean NTL

In [ ]:
def mean_ntl(tif_path: Path) -> float:
    with rasterio.open(tif_path) as src:
        arr = src.read(1).astype("float32")
        nodata = src.nodata
        if nodata is not None:
            arr[arr == nodata] = np.nan
        return float(np.nanmean(arr))

imgs["mean_ntl"] = imgs["path"].apply(mean_ntl)
print(f"Mean NTL calculated for {len(imgs)} images")

# 7) Event availability summary

In [ ]:
event_availability = (
    imgs.groupby("event_key")
        .agg(
            county=("county", "first"),
            event_id=("event_id", "first"),
            outage_start=("outage_start", "first"),
            outage_end=("outage_end", "first"),
            n_pre=("phase", lambda x: (x == "pre").sum()),
            n_during=("phase", lambda x: (x == "during").sum()),
            n_post=("phase", lambda x: (x == "post").sum()),
            total_images=("date", "count"),
            avg_snow_frac=("snow_frac", "mean"),
            max_snow_frac=("snow_frac", "max"),
        )
        .reset_index()
)

event_availability.to_csv(RESULT / "event_availability.csv", index=False)
print("Event availability:")
display(event_availability)

# 8) Event x Phase statistics

In [ ]:
event_phase = (
    imgs.groupby(["event_key", "phase"])
        .agg(
            county=("county", "first"),
            event_id=("event_id", "first"),
            min_ntl=("mean_ntl", "min"),
            mean_ntl=("mean_ntl", "mean"),
            n_images=("mean_ntl", "count"),
            phase_start=("date", "min"),
            phase_end=("date", "max"),
            outage_start=("outage_start", "first"),
            outage_end=("outage_end", "first"),
            avg_snow_frac=("snow_frac", "mean"),
        )
        .reset_index()
        .sort_values(["event_key", "phase"])
)

event_phase.to_csv(RESULT / "event_phase_stats.csv", index=False)
print("Event x Phase stats:")
display(event_phase)

# 9) Relative NTL change analysis

In [ ]:
print("="*60)
print("Relative NTL Change Analysis (during vs pre)")
print("="*60)

results = []
for evt in event_phase["event_key"].unique():
    evt_df = event_phase[event_phase["event_key"] == evt]
    
    pre = evt_df[evt_df["phase"] == "pre"]
    during = evt_df[evt_df["phase"] == "during"]
    
    if len(pre) > 0 and len(during) > 0:
        pre_ntl = pre["mean_ntl"].values[0]
        during_ntl = during["mean_ntl"].values[0]
        pre_snow = pre["avg_snow_frac"].values[0]
        during_snow = during["avg_snow_frac"].values[0]
        
        change_pct = ((during_ntl - pre_ntl) / pre_ntl) * 100
        
        results.append({
            "event_key": evt,
            "county": pre["county"].values[0],
            "pre_ntl": round(pre_ntl, 2),
            "during_ntl": round(during_ntl, 2),
            "change_pct": round(change_pct, 1),
            "pre_snow_frac": round(pre_snow, 3),
            "during_snow_frac": round(during_snow, 3),
            "n_pre": pre["n_images"].values[0],
            "n_during": during["n_images"].values[0],
        })

change_df = pd.DataFrame(results)
print("\nNTL Change:")
display(change_df)

print(f"\nSummary:")
print(f"  Events with NTL decrease: {(change_df['change_pct'] < 0).sum()} / {len(change_df)}")
print(f"  Average change: {change_df['change_pct'].mean():.1f}%")
print(f"  Events with high snow (>10%): {((change_df['pre_snow_frac'] > 0.1) | (change_df['during_snow_frac'] > 0.1)).sum()}")

change_df.to_csv(RESULT / "ntl_change_analysis.csv", index=False)

# 10) Snow impact analysis

In [ ]:
print("="*60)
print("Snow Impact Analysis")
print("="*60)

# Correlation between snow and NTL
corr = imgs[["snow_frac", "mean_ntl"]].corr().iloc[0, 1]
print(f"\nCorrelation (snow_frac vs mean_ntl): {corr:.3f}")

# Snow by phase
print("\nSnow fraction by phase:")
snow_by_phase = imgs.groupby("phase")["snow_frac"].agg(["mean", "std", "max", "count"])
print(snow_by_phase)

# High snow events
high_snow_events = change_df[
    (change_df["pre_snow_frac"] > 0.1) | (change_df["during_snow_frac"] > 0.1)
]
if len(high_snow_events) > 0:
    print(f"\nHigh snow events (snow_frac > 10%):")
    display(high_snow_events[["event_key", "change_pct", "pre_snow_frac", "during_snow_frac"]])
else:
    print("\nNo high snow events found.")

# 11) Visualization: Time series

In [ ]:
ts = (
    imgs.groupby(["event_key", "date"])
        .agg(
            mean_ntl=("mean_ntl", "mean"),
            snow_frac=("snow_frac", "mean"),
            county=("county", "first"),
            event_id=("event_id", "first"),
            outage_start=("outage_start", "first"),
            outage_end=("outage_end", "first"),
        )
        .reset_index()
        .sort_values(["event_key", "date"])
)

for event_key, g in ts.groupby("event_key"):
    g = g.sort_values("date")
    if g.empty:
        continue
    
    start = g["outage_start"].iloc[0]
    end = g["outage_end"].iloc[0]
    county = g["county"].iloc[0]
    event_id = g["event_id"].iloc[0]
    max_snow = g["snow_frac"].max()

    fig, ax1 = plt.subplots(figsize=(10, 5))
    
    # NTL line
    color1 = "tab:blue"
    ax1.plot(g["date"], g["mean_ntl"], marker="o", linewidth=2, color=color1, label="Mean NTL")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Mean NTL (nW/cm2/sr)", color=color1)
    ax1.tick_params(axis="y", labelcolor=color1)
    
    # Snow fraction on secondary axis (if any snow)
    if max_snow > 0.01:
        ax2 = ax1.twinx()
        color2 = "tab:cyan"
        ax2.fill_between(g["date"], 0, g["snow_frac"], alpha=0.3, color=color2, label="Snow frac")
        ax2.set_ylabel("Snow Fraction", color=color2)
        ax2.tick_params(axis="y", labelcolor=color2)
        ax2.set_ylim(0, max(0.5, max_snow * 1.2))
    
    # Outage period
    if pd.notna(start) and pd.notna(end):
        ax1.axvspan(start, end, alpha=0.2, color="red", label="Outage period")

    ax1.set_title(f"{county} | {event_id}\n{event_key}")
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc="upper left")
    plt.xticks(rotation=45)
    plt.tight_layout()

    fname = f"{event_key}_timeseries.png"
    plt.savefig(PLOT_DIR / fname, dpi=200, bbox_inches="tight")
    plt.close()

print(f"Saved time series plots to: {PLOT_DIR}")

# 12) POI Buffer Analysis Setup

In [ ]:
# POI data path
POI_CSV = DATA / "raw" / "POI" / "texas_critical_infra_points_2022.csv"

poi = pd.read_csv(POI_CSV)
poi = poi.rename(columns={"lon": "longitude", "lat": "latitude"})

gdf_poi = gpd.GeoDataFrame(
    poi,
    geometry=gpd.points_from_xy(poi["longitude"], poi["latitude"]),
    crs="EPSG:4326"
)

# Project to metric CRS (Texas Centric Albers)
gdf_poi = gdf_poi.to_crs(epsg=3083)

# 1000m buffer
BUFFER_M = 1000
gdf_poi["geometry"] = gdf_poi.geometry.buffer(BUFFER_M)

# Map city_source to county name
CITY_TO_COUNTY = {
    "Houston": "Harris",
    "Dallas": "Dallas",
}
gdf_poi["county"] = gdf_poi["city_source"].map(CITY_TO_COUNTY)
gdf_poi = gdf_poi[gdf_poi["county"].notna()].copy()

print(f"POI count: {len(gdf_poi)}")
print(f"Counties: {gdf_poi['county'].unique()}")

# 13) Calculate Buffer vs Non-buffer NTL

In [ ]:
def zonal_nanmean(src, geom):
    """Calculate nanmean within geometry"""
    try:
        out, _ = mask(src, [geom], crop=False, all_touched=True)
        arr = out[0].astype("float32")
        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan
        return float(np.nanmean(arr))
    except Exception:
        return np.nan


# Create buffer union for each county
buffers_by_county = {
    c: gdf_poi[gdf_poi["county"] == c].geometry.unary_union
    for c in gdf_poi["county"].unique()
}

print(f"Buffer counties: {list(buffers_by_county.keys())}")

records = []
error_count = 0

for idx, r in imgs.iterrows():
    county = r["county"]
    tif = r["path"]

    if county not in buffers_by_county:
        continue

    try:
        with rasterio.open(tif) as src:
            extent = box(*src.bounds)

            # Project buffer to TIF's CRS
            buf_geom = gpd.GeoSeries(
                [buffers_by_county[county]], 
                crs=gdf_poi.crs
            ).to_crs(src.crs).iloc[0]

            # Non-facility area = extent - buffer
            nonbuf_geom = extent.difference(buf_geom)

            buf_mean = zonal_nanmean(src, buf_geom)
            nonbuf_mean = zonal_nanmean(src, nonbuf_geom) if not nonbuf_geom.is_empty else np.nan

        rec = dict(r)
        rec["mean_ntl_buffer"] = buf_mean
        rec["mean_ntl_nonbuffer"] = nonbuf_mean
        records.append(rec)

    except Exception as e:
        print(f"Error: {tif.name}: {e}")
        error_count += 1

print(f"\nProcessed: {len(records)}, Errors: {error_count}")

df_buf = pd.DataFrame(records)

# 14) Buffer vs Non-buffer Statistics

In [ ]:
event_phase_buffer = (
    df_buf.groupby(["event_key", "phase"])
          .agg(
              county=("county", "first"),
              buffer_min=("mean_ntl_buffer", "min"),
              buffer_mean=("mean_ntl_buffer", "mean"),
              nonbuffer_min=("mean_ntl_nonbuffer", "min"),
              nonbuffer_mean=("mean_ntl_nonbuffer", "mean"),
              n_images=("mean_ntl_buffer", "count"),
              avg_snow_frac=("snow_frac", "mean"),
              outage_start=("outage_start", "first"),
              outage_end=("outage_end", "first"),
          )
          .reset_index()
)

event_phase_buffer.to_csv(RESULT / "event_phase_buffer_vs_nonbuffer.csv", index=False)
print("Event x Phase x Buffer stats:")
display(event_phase_buffer)

# 15) Buffer vs Non-buffer Change Analysis

In [ ]:
print("="*60)
print("Buffer vs Non-buffer Relative Change (during vs pre)")
print("="*60)

buf_results = []
for evt in event_phase_buffer["event_key"].unique():
    evt_df = event_phase_buffer[event_phase_buffer["event_key"] == evt]
    
    pre = evt_df[evt_df["phase"] == "pre"]
    during = evt_df[evt_df["phase"] == "during"]
    
    if len(pre) > 0 and len(during) > 0:
        pre_buf = pre["buffer_mean"].values[0]
        during_buf = during["buffer_mean"].values[0]
        pre_nonbuf = pre["nonbuffer_mean"].values[0]
        during_nonbuf = during["nonbuffer_mean"].values[0]
        
        buf_change = ((during_buf - pre_buf) / pre_buf) * 100 if pre_buf > 0 else np.nan
        nonbuf_change = ((during_nonbuf - pre_nonbuf) / pre_nonbuf) * 100 if pre_nonbuf > 0 else np.nan
        
        buf_results.append({
            "event_key": evt,
            "county": pre["county"].values[0],
            "buf_change_pct": round(buf_change, 1) if not np.isnan(buf_change) else np.nan,
            "nonbuf_change_pct": round(nonbuf_change, 1) if not np.isnan(nonbuf_change) else np.nan,
            "diff": round(buf_change - nonbuf_change, 1) if not (np.isnan(buf_change) or np.isnan(nonbuf_change)) else np.nan,
            "pre_snow": round(pre["avg_snow_frac"].values[0], 3),
            "during_snow": round(during["avg_snow_frac"].values[0], 3),
        })

buf_change_df = pd.DataFrame(buf_results)
print("\nBuffer vs Non-buffer change:")
display(buf_change_df)

print(f"\nInterpretation:")
print(f"  diff > 0: Buffer dropped LESS than non-buffer (facilities more resilient)")
print(f"  diff < 0: Buffer dropped MORE than non-buffer")
print(f"\nEvents where buffer more resilient: {(buf_change_df['diff'] > 0).sum()} / {len(buf_change_df)}")

buf_change_df.to_csv(RESULT / "buffer_change_analysis.csv", index=False)

# 16) Buffer vs Non-buffer Time Series Plots

In [ ]:
PLOT_DIR_BUF = RESULT / "plots_buffer"
PLOT_DIR_BUF.mkdir(parents=True, exist_ok=True)

ts_buf = (
    df_buf.groupby(["event_key", "date"])
          .agg(
              buffer=("mean_ntl_buffer", "mean"),
              nonbuffer=("mean_ntl_nonbuffer", "mean"),
              snow_frac=("snow_frac", "mean"),
              county=("county", "first"),
              event_id=("event_id", "first"),
              outage_start=("outage_start", "first"),
              outage_end=("outage_end", "first"),
          )
          .reset_index()
          .sort_values(["event_key", "date"])
)

plot_count = 0
for event_key, g in ts_buf.groupby("event_key"):
    g = g.sort_values("date")
    if g.empty or g["buffer"].isna().all():
        continue

    start = g["outage_start"].iloc[0]
    end = g["outage_end"].iloc[0]
    county = g["county"].iloc[0]
    event_id = g["event_id"].iloc[0]

    fig, ax1 = plt.subplots(figsize=(10, 5))
    
    ax1.plot(g["date"], g["buffer"], marker="o", linewidth=2, 
             color="tab:orange", label="Critical facility buffer (1km)")
    ax1.plot(g["date"], g["nonbuffer"], marker="s", linewidth=2, 
             color="tab:blue", label="Non-facility area")
    
    if pd.notna(start) and pd.notna(end):
        ax1.axvspan(start, end, alpha=0.2, color="red", label="Outage period")

    ax1.set_xlabel("Date")
    ax1.set_ylabel("Mean NTL (nW/cm2/sr)")
    ax1.set_title(f"{county} | {event_id}\n{event_key}\nBuffer vs Non-buffer")
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc="upper left")
    plt.xticks(rotation=45)
    plt.tight_layout()

    fname = f"{event_key}_buffer_vs_nonbuffer.png"
    plt.savefig(PLOT_DIR_BUF / fname, dpi=200, bbox_inches="tight")
    plt.close()
    plot_count += 1

print(f"Saved {plot_count} buffer plots to: {PLOT_DIR_BUF}")